# Testing models

In [49]:
import sys
sys.path.append('src/')
import pandas as pd
from Models import deberta_base_nli, bart_nli, deberta_v3_nli
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from Helpers import *

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

## Basic tests of models

In [2]:
deberta_base_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

({'contradiction': 96.9, 'neutral': 3.0, 'entailment': 0.0}, ['contradiction'])

In [3]:
bart_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

({'contradiction': 93.7, 'neutral': 5.8, 'entailment': 0.5}, ['contradiction'])

In [4]:
deberta_v3_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

/usr/local/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


({'contradiction': 97.7, 'neutral': 2.2, 'entailment': 0.1}, ['contradiction'])

# Testing models with random samples from toxic dataset

In [53]:
df = get_random_samples("data/toxicbias_train.csv", 50)

## Calculate labels

In [47]:
import pandas as pd

def add_nli_predictions(df, model):
    df = df.copy()  # Create a copy of the DataFrame

    df = df.apply(lambda row: predict_labels(row, model), axis=1)

    return df

def predict_labels(row, model):
    comment_text = row['comment_text']
    bias = row['bias']
    rationale = row['rationale']

    if bias == 'neutral' or pd.isna(rationale):
        rationale = "Bias or prejudice in the text."

    scores, labels = model(comment_text, rationale)
    row['predicted_label_rationale'] = labels
    row['predicted_scores_rationale'] = scores

    scores, labels = model(comment_text, "Bias or prejudice in the text.")
    row['predicted_label_base'] = labels
    row['predicted_scores_base'] = scores

    return row

In [55]:
df_deberta_base = add_nli_predictions(df, deberta_base_nli)
df_bart_large = add_nli_predictions(df, bart_nli)
df_deberta_v3 = add_nli_predictions(df, deberta_v3_nli)

/usr/local/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Calculate accuracy of labels

In [78]:
def label_breakdown(df, label_column_name = 'predicted_label_rationale'):
    label_breakdown = {
        'bias': {'contradiction': 0, 'neutral': 0, 'entailment': 0},
        'neutral': {'contradiction': 0, 'neutral': 0, 'entailment': 0}
    }

    for _, row in df.iterrows():
        stance = row['bias']
        predicted_label_rationale = row[label_column_name][0]
        label_breakdown[stance][predicted_label_rationale] += 1

    return label_breakdown['bias'], label_breakdown['neutral']


In [79]:
label_breakdown(df_deberta_base)

({'contradiction': 3, 'neutral': 34, 'entailment': 3},
 {'contradiction': 0, 'neutral': 1, 'entailment': 9})

In [65]:
df_deberta_base

,comment_text,identity_attack_x,bias,target,category,rationale,predicted_label_rationale,predicted_scores_rationale,predicted_label_base,predicted_scores_base
2104,At least I am not Rachel Dolzeal type appropriating an Indian name to covered for your liberal whiteness. There is nothing to address when you are dealing with someone who just like to type for typing sakes making it worst that anything you write is full of donkeys waste. My apologizes too all the donkeys in the world for that politically insensitive comments,0.500000,bias,liberals,political,prejudice against liberals,[contradiction],"{'contradiction': 94.7, 'neutral': 5.1, 'entailment': 0.2}",[contradiction],"{'contradiction': 92.6, 'neutral': 6.3, 'entailment': 1.1}"
1558,The GOP is just a white identity party. They are not conservative. They love big government as long as it attacks none white people and uppity women. They have recruited white racists successfully. Now we all can see who and what they are. They care more about being white than being American or human. They are proud in their ignorance. That works great for Trump. He loves the poorly educated because they are so easy to con.,0.600000,bias,"whites,women","race,gender",prejudice against whites and women,[neutral],"{'contradiction': 4.9, 'neutral': 88.7, 'entailment': 6.4}",[entailment],"{'contradiction': 1.1, 'neutral': 3.8, 'entailment': 95.1}"
1244,Why is the DP so reluctant to call this guy what he is an allAmerican white boy TERRORIST? Does the DP think it is genetically impossible for a white guy top be a terrorist ot is there something more racially sinister at work here?,0.500000,neutral,NaN,none,NaN,[entailment],"{'contradiction': 0.4, 'neutral': 6.1, 'entailment': 93.6}",[entailment],"{'contradiction': 0.4, 'neutral': 6.1, 'entailment': 93.6}"
1830,They didnt explicitly use the word God. They used the far more anodyne term Creator. Ooops Your religious crutch is not important to me. Nor your bigoted comments denigrating atheists.,0.700000,bias,atheists,religion,prejudice towards atheists,[entailment],"{'contradiction': 2.5, 'neutral': 31.2, 'entailment': 66.3}",[entailment],"{'contradiction': 0.6, 'neutral': 4.3, 'entailment': 95.1}"
2137,Louise Linton publicly providing proof you dont have to be poor to be white trash.,0.769231,bias,whites,race,prejudice against whites,[neutral],"{'contradiction': 14.9, 'neutral': 68.4, 'entailment': 16.7}",[entailment],"{'contradiction': 9.1, 'neutral': 20.2, 'entailment': 70.7}"
236,And you probably think it is ok to lightly beat women.,0.257576,neutral,NaN,none,NaN,[entailment],"{'contradiction': 0.6, 'neutral': 6.1, 'entailment': 93.3}",[entailment],"{'contradiction': 0.6, 'neutral': 6.1, 'entailment': 93.3}"
576,They should all be given long sentences and thrown in prison like they did during the crack epidemic but of course that situation involved a bunch o black and brown people so now its treatment sympathy and a public health emergency. You got to love America...that great Christian Nation. Unbelievable,0.500000,bias,"blacks,christians","race,religion",prejudice against blacks and christians,[contradiction],"{'contradiction': 47.3, 'neutral': 43.1, 'entailment': 9.7}",[entailment],"{'contradiction': 1.5, 'neutral': 3.0, 'entailment': 95.4}"
3641,Also Remember Catholic laity are amongst the most supportive of civil marriage equality. If so then this Catholic laity is facilitating one of the worst violations of Catholic teaching and promoting one of the greatest hoaxes on mankind which is that the sexual union of two gays can be a marriage.,0.500000,bias,"christians,gays","religion,lgbtq",prejudice against christians and gays,[neutral],"{'contradiction': 18.9, 'neutral': 75.6, 'entailment': 5.5}",[contradiction],"{'contradiction': 53.3, 'neutral': 8.9, 'entailment': 37.8}"
2710,I find it odd that people think they can negotiate with a religion that wants to cut your head off because youre a Christian.,0.600000,bias,christians,religion,prejudice agai